In [ ]:
import pandas as pd
import numpy as np
import urllib.request
import io
url = "http://motioncore-umh.cs.umn.edu/4980/college.csv"
data = urllib.request.urlopen(url)
csv_bytes = data.read()
csv_string = csv_bytes.decode("utf8") #TODO: it's bad practice to assume the data is utf8 formated
csv_data = io.StringIO(csv_string) #Padas expects CSV from a file buffer, not a raw string
df = pd.read_csv(csv_data)
names = list(df.columns)

df["selective"] = df['Top10perc'] > 50
df

,School,Private,Apps,Accept,Enroll,Top10perc,Top25perc,F.Undergrad,P.Undergrad,Outstate,Room.Board,Books,Personal,PhD,Terminal,S.F.Ratio,perc.alumni,Expend,Grad.Rate,selective
0,Abilene Christian University,Yes,1660,1232,721,23,52,2885,537,7440,3300,450,2200,70,78,18.1,12,7041,60,False
1,Adelphi University,Yes,2186,1924,512,16,29,2683,1227,12280,6450,750,1500,29,30,12.2,16,10527,56,False
2,Adrian College,Yes,1428,1097,336,22,50,1036,99,11250,3750,400,1165,53,66,12.9,30,8735,54,False
3,Agnes Scott College,Yes,417,349,137,60,89,510,63,12960,5450,450,875,92,97,7.7,37,19016,59,True
4,Alaska Pacific University,Yes,193,146,55,16,44,249,869,7560,4120,800,1500,76,72,11.9,2,10922,15,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
772,Worcester State College,No,2197,1515,543,4,26,3089,2029,6797,3900,500,1200,60,60,21.0,14,4469,40,False
773,Xavier University,Yes,1959,1805,695,24,47,2849,1107,11520,4960,600,1250,73,75,13.3,31,9189,83,False
774,Xavier University of Louisiana,Yes,2097,1915,695,34,61,2793,166,6900,4200,617,781,67,75,14.4,20,8323,49,False
775,Yale University,Yes,10705,2453,1317,95,99,5217,83,19840,6510,630,2115,96,96,5.8,49,40386,99,True


In [ ]:
import seaborn as sns
sns.set(font_scale=1.2)

In [ ]:
sns.pairplot(df)


In [ ]:
sns.pairplot(df,hue="selective")

Two features for KNN classifier : Outstate, Expend

In [ ]:
#Choose which data is featuers (X) and labels (y)
X = df[ ['Outstate','Expend', 'Room.Board', 'perc.alumni'] ].to_numpy()
y = df['selective'].to_numpy()

In [ ]:
def train_test_split(X,y,test_size):
  numRows = y.size                     #number or rows in the entire dataset
  splitPoint = int((1-test_size)*numRows)  #the row index test_size% of the way through
  p = np.random.permutation(numRows)   #array for shuffling data
  X = X[p]
  y = y[p]

  #Training data array
  X_train = X[:splitPoint]  # training features
  y_train = y[:splitPoint]  # training labels

  #Testing data array
  X_test = X[splitPoint:]  # testing features
  y_test = y[splitPoint:] # testing labels

  return X_train, X_test, y_train, y_test

In [ ]:
def accuracy(classify, X_train, y_train, X_test, y_test):
  pred = classify(X_train, y_train, X_test)
  correct_pred = (pred == y_test)
  total_preds = y_test.size
  return correct_pred.sum()/total_preds

In [ ]:
from sklearn import neighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
def knn_classifier(X_train, y_train, X_test):
  # instantiate learning model and fit data
  knn = KNeighborsClassifier(n_neighbors=k)
  knn.fit(X_train, y_train)

  # predict the response
  pred = knn.predict(X_test)
  return pred

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X,y,0.33)


def evaluate_model(y_true, y_pred, prefix=""):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    print(f"{prefix}Accuracy: {accuracy:.2f}")
    print(f"{prefix}Precision: {precision:.2f}")
    print(f"{prefix}Recall: {recall:.2f}")
    print(f"{prefix}F1 Score: {f1:.2f}")

In [ ]:
# Very low k value
k = 1
# print(knn_classifier(X_train, y_train, X_test))
evaluate_model(y_test, knn_classifier(X_train, y_train, X_test))

In [ ]:
# Very large k value
k = 50
# print(knn_classifier(X_train, y_train, X_test))
evaluate_model(y_test, knn_classifier(X_train, y_train, X_test))

In [ ]:
# The classifier that says that all of the schools are selective

# On the training data, the KNN classifier with a small K performs better
# because it tends to fit the training data more closely, resulting in better precision and recall.
# The classifier that predicts all schools as highly selective may have a high recall but low precision,
# while the one that predicts no schools as highly selective will have high precision but low recall.

k = 1
# print(knn_classifier(X_train, y_train, X_test))
evaluate_model(y_test, knn_classifier(X_train, y_train, X_test))

In [ ]:
# The classifier that says that none of the schools are selective
k = 100
# print(knn_classifier(X_train, y_train, X_test))
evaluate_model(y_test, knn_classifier(X_train, y_train, X_test))

In [ ]:
# KNN analysis on the hypothetical school for Ques. 5
k = 5
X_test_hypt = np.array([[14000, 1200, 5000, 30]])
y_test_hypt = np.array([[True]])
evaluate_model(y_test_hypt, knn_classifier(X_train, y_train, X_test_hypt))

Accuracy: 0.00
Precision: 0.00
Recall: 0.00
F1 Score: 0.00


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
